In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

### Este notebook tem como objetivo praticar o uso do KNN no contexto de classificação e do ensemble RandomForestClassifier.
<br>
<br>
Para este trabalho estaremos utilizando um dataset referente a autenticação de notas bancárias.
<br>
<br>
Em sua construção foram utilizadas imagens capturadas de 1372 notas verdadeiras e falsas e através de uma ferramenta foram compactadas em onduletas que geraram os dados encontrados logo a seguir.
<br>
<br>
Para mais informações sobre o data set clique: <p><a href="https://www.openml.org/d/1462">Aqui! <p> 

In [2]:
df = pd.read_csv("notas_bancarias.csv")

In [3]:
df

,V1,V2,V3,V4,Class
0,3.62160,8.66610,-2.8073,-0.44699,1
1,4.54590,8.16740,-2.4586,-1.46210,1
2,3.86600,-2.63830,1.9242,0.10645,1
3,3.45660,9.52280,-4.0112,-3.59440,1
4,0.32924,-4.45520,4.5718,-0.98880,1
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,2
1368,-1.38870,-4.87730,6.4774,0.34179,2
1369,-3.75030,-13.45860,17.5932,-2.77710,2
1370,-3.56370,-8.38270,12.3930,-1.28230,2


In [4]:
#Conferindo algumas infos:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      1372 non-null   float64
 1   V2      1372 non-null   float64
 2   V3      1372 non-null   float64
 3   V4      1372 non-null   float64
 4   Class   1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


As classes 1 são notas comuns. As classes 2 são notas forjadas. Vamos utilizar aprendizagem supervisionada no contexto de classificação para tentar aprender a identificá-las.

In [5]:
#Vamos dividir em treino e teste nossos dados:

#Preparando as variáveis:

dff = df.copy() #Criando uma cópia do df.
dff = dff.drop(columns=["Class"]) #dropando a target do dataframe copiado.
dff

,V1,V2,V3,V4
0,3.62160,8.66610,-2.8073,-0.44699
1,4.54590,8.16740,-2.4586,-1.46210
2,3.86600,-2.63830,1.9242,0.10645
3,3.45660,9.52280,-4.0112,-3.59440
4,0.32924,-4.45520,4.5718,-0.98880
...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949
1368,-1.38870,-4.87730,6.4774,0.34179
1369,-3.75030,-13.45860,17.5932,-2.77710
1370,-3.56370,-8.38270,12.3930,-1.28230


In [6]:
X = dff.values #Atribuindo a X nossas variáveis preditivas:
y = df.Class.values.reshape(-1,1) #Atribuindo a y nossa target.

#Conferindo:

print(X)
print(y)

[[  3.6216    8.6661   -2.8073   -0.44699]
 [  4.5459    8.1674   -2.4586   -1.4621 ]
 [  3.866    -2.6383    1.9242    0.10645]
 ...
 [ -3.7503  -13.4586   17.5932   -2.7771 ]
 [ -3.5637   -8.3827   12.393    -1.2823 ]
 [ -2.5419   -0.65804   2.6842    1.1952 ]]
[[1]
 [1]
 [1]
 ...
 [2]
 [2]
 [2]]


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
#Dividindo os dados:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.30) #30% para os dados de teste
print(X_train.shape)
print(y_train.shape)

(960, 4)
(960, 1)


Com os dados divididos vamos começar a trabalhar com a classificação.
<br>
Vamos utilizar um gridsearch com alguns algoritmos para encontrar os melhores parâmetros:

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, confusion_matrix

In [10]:
#Utilizando o ensemble RandomForest:

e_base = RandomForestClassifier()

param_tree = {"max_depth":range(1,16),"max_leaf_nodes":range(1,5)} #Definindo alguns parâmetros para o gridsearch:

grid = GridSearchCV(e_base, param_tree, scoring = "f1", cv=10) #Avaliando inicialmente com a métrica f1:

In [11]:
#Fit:

grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 16),
                         'max_leaf_nodes': range(1, 5)},
             scoring='f1')

In [12]:
#Conferindo os melhores parâmetros:

grid.best_params_

{'max_depth': 14, 'max_leaf_nodes': 4}

In [13]:
#Conferindo a melhor nota de f1:

grid.best_score_

0.9424097754714221

In [14]:
#Utilizando KNN no gridsearch:

e_base_k = KNeighborsClassifier()
param_knn = {"n_neighbors":range(1,9)}
grid = GridSearchCV(e_base_k, param_knn, scoring = "f1", cv=10)

In [15]:
#Fit

grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 9)}, scoring='f1')

In [16]:
#Melhor parâmetro:

grid.best_params_

{'n_neighbors': 4}

In [17]:
#Melhor score de f1:

grid.best_score_

1.0

Vemos que a nota de f1 com relação ao KNN indica um overfitting, vamos tirar a limpo no nosso teste final ambos os algoritmos:

In [18]:
#Começando pelo random forest:

rand_forest = RandomForestClassifier(max_depth =14, max_leaf_nodes= 4)

In [19]:
rand_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=14, max_leaf_nodes=4)

In [20]:
#Salvando numa variável separada as predições feitas nas variáveis explicativas do dataset de teste:

y_predict = rand_forest.predict(X_test)

In [21]:
f1_score(y_true = y_test, y_pred = y_predict)

0.9270386266094421

In [22]:
confusion_matrix(y_true = y_test, y_pred = y_predict)

array([[216,   9],
       [ 25, 162]], dtype=int64)

Podemos perceber que a nota encontrada nos dados de teste continua próxima a encontrada no treino. A nossa predição retornou um bom valor de f1, mas que nesse caso específico tem pormenores. Na identificação de fraudes a confusion_matrix nos mostra que temos 25 falsos positivos e 9 falsos negativos. Significa dizer que neste universo possível, 25 são alarmes falsos, ou seja, não são notas falsas, e 9 eram fraudes e não foram detectadas.
<br>
<br> 
A medida que o algoritmo fosse sendo alimentado com mais dados alcançaríamos aprendizagens melhores. No entanto o resultado é bem satisfatório para um universo pequeno de informações.

Vamos testar o KNN para identificarmos se houve realmente um overfitting:

In [23]:
knn = KNeighborsClassifier(n_neighbors = 4)

In [24]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=4)

In [25]:
y_knn_pred = knn.predict(X_test)

In [26]:
f1_score(y_true = y_test, y_pred = y_knn_pred)

1.0

In [27]:
confusion_matrix(y_true = y_test, y_pred = y_knn_pred)

array([[225,   0],
       [  0, 187]], dtype=int64)

Diferente do que pensávamos o KNN não "overfitou" os dados de treino, ele de fato pode aprender. Isso é perceptível pelos fit ter sido nos dados de treinamento e com a aprendizagem nestes dados o algoritmo foi capaz de fazer a predição com perfeição nos dados de teste, sendo ainda melhor que os resultados no Ensemble de árvores. 